<a href="https://colab.research.google.com/github/npuch/Compiler/blob/master/PandasSetDeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Drive


In [2]:
  from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch

Kaggle -> Google Drive -> Colab

In [4]:
!git clone https://github.com/scaleapi/pandaset-devkit.git
!pip install pandaset-devkit/python/

Cloning into 'pandaset-devkit'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 191 (delta 47), reused 22 (delta 7), pack-reused 96
Receiving objects: 100% (191/191), 21.05 MiB | 33.01 MiB/s, done.
Resolving deltas: 100% (71/71), done.
Processing ./pandaset-devkit/python
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 151 kB 7.3 MB/s 
     |████████████████████████████████| 156 kB 35.7 MB/s 
     |████████████████████████████████| 115 kB 43.5 MB/s 
     |████████████████████████████████| 58 kB 6.1 MB/s 


In [5]:
from pandaset import DataSet

dataset = DataSet('/content/drive/My Drive/Truckin/')

print(dataset)

print("The sequences that have (cuboid + semantic) segmentation annotations:\n", dataset.sequences(with_semseg=True))

for i in dataset.sequences(with_semseg=False):
    print(i)

The sequences that have (cuboid + semantic) segmentation annotations:
 ['005', '002', '003', '001']
005
006
002
004
003
008
001


Move Data from drive to dataset object

In [ ]:
seq002 = dataset['002']

#load into memory
seq002.load()

print(seq002.__dict__)

Now that we have seen how the seq and data set look like we need to access the lidar data

In [ ]:
lidarObj = seq002.lidar

print(lidarObj)

print(f"Time stamps from lidar object: {lidarObj.timestamps}")

#print(f"Panda set object: {pandaset.sensors.Lidar}")

print(f"Lidar data obj {lidarObj.data}")



Lidar to Pytorch Var

In [ ]:
import plotly.graph_objects as go 
  
headerColor = 'rgb(116,0,4)'

annon_class = []
for key in seq002.semseg.classes:
    annon_class.append(seq002.semseg.classes[key])
  
fig = go.Figure(data=[go.Table( 
    header = dict(values=['<b>Semantic Segmentation Annotations for Sequence 002 (Possible Labels)</b>'],
                  line_color='darkslategray',
                  fill_color=headerColor,
                  align=['left','center'],
                  font=dict(color='white', size=12)
                 ), 
    cells = dict(values=[annon_class],
                 line_color='darkslategray',
                 align = ['left', 'center'],
                 font = dict(color = headerColor, size = 11)
                )) 
                     ]) 
fig.show() 

In [ ]:
lidar_poses = seq002.lidar._poses
print(lidar_poses[0])

In [ ]:
import csv
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandaset

import os
   
pfile = "/content/drive/My Drive/Truckin/001/lidar/01.pkl"
with open(pfile, "rb") as fin:
    data = pickle.load(fin)

data.info()

dataset = pandaset.DataSet("/content/drive/My Drive/Truckin/")
seq002 = dataset["002"]
seq002.load_lidar().load_semseg()

points3d_lidar_xyz = seq002.lidar.data

print(seq002.semseg.classes)


3d plots of lidar

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import random

ax = plt.axes(projection='3d')

xyz = data[['x', 'y', 'z']].to_numpy()
color_maps = [(random.random(), random.random(), random.random()) for _ in range(1000 + 1)]
ax.scatter(xyz[:,0], xyz[:,1], xyz[:,2])

Lidar Data to pytorch var

In [ ]:
!pip install torch
!echo torch version 

In [ ]:
import torch
import numpy as np


xyz = data[['x', 'y', 'z']].to_numpy()
print(xyz)
print(xyz.shape)


xyz_torch = torch.from_numpy(xyz)
print(xyz_torch)
print(type(xyz_torch))
print(xyz_torch.size())



Goal: Train model one frame at a time

In [ ]:
import torch
import numpy as np


lidarList = ['/00.pkl', '/01.pkl','/02.pkl','/03.pkl','/04.pkl','/05.pkl','/06.pkl','/07.pkl', 
             '/08.pkl','/09.pkl','/10.pkl','/11.pkl', '/11.pkl','/12.pkl','/13.pkl','/14.pkl',
             '/15.pkl','/16.pkl','/17.pkl','/18.pkl','/19.pkl','/20.pkl','/21.pkl','/22.pkl',
             '/23.pkl','/24.pkl', '/25.pkl','/26.pkl','/27.pkl','/28.pkl','/29.pkl', 
             '/30.pkl', '/31.pkl','/32.pkl','/33.pkl','/34.pkl','/35.pkl','/36.pkl','/37.pkl', '/38.pkl','/39.pkl',
             '/40.pkl', '/41.pkl','/42.pkl','/43.pkl','/44.pkl','/45.pkl','/46.pkl','/47.pkl', '/48.pkl','/49.pkl',
             '/50.pkl', '/51.pkl','/52.pkl','/53.pkl','/54.pkl','/55.pkl','/56.pkl','/57.pkl', '/58.pkl','/59.pkl',
             '/60.pkl', '/61.pkl','/62.pkl','/63.pkl','/64.pkl','/65.pkl','/66.pkl','/67.pkl', '/68.pkl','/69.pkl',
             '/70.pkl', '/71.pkl','/72.pkl','/73.pkl','/74.pkl','/75.pkl','/76.pkl','/77.pkl', '/78.pkl','/79.pkl']

for lidarFile in lidarList:
    pfile = "/content/drive/My Drive/Truckin/001/lidar" + lidarFile
    with open(pfile, "rb") as fin:
        data = pickle.load(fin)

    data.info()

    xyz = data[['x', 'y', 'z']].to_numpy()
    print(xyz)
    print(xyz.shape)


    xyz_torch = torch.from_numpy(xyz)
    print(xyz_torch)
    print(type(xyz_torch))
    print(xyz_torch.size())
    

Get labels

In [ ]:
pfile = "/content/drive/MyDrive/Truckin/002/annotations/cuboids/01.pkl"
with open(pfile, "rb") as fin:
    data = pickle.load(fin)

data.info() #Get data dictionary

import torch
import numpy as np


annotationList = ['/00.pkl', '/01.pkl','/02.pkl','/03.pkl','/04.pkl','/05.pkl','/06.pkl','/07.pkl', 
             '/08.pkl','/09.pkl','/10.pkl','/11.pkl', '/11.pkl','/12.pkl','/13.pkl','/14.pkl',
             '/15.pkl','/16.pkl','/17.pkl','/18.pkl','/19.pkl','/20.pkl','/21.pkl','/22.pkl',
             '/23.pkl','/24.pkl', '/25.pkl','/26.pkl','/27.pkl','/28.pkl','/29.pkl', 
             '/30.pkl', '/31.pkl','/32.pkl','/33.pkl','/34.pkl','/35.pkl','/36.pkl','/37.pkl', '/38.pkl','/39.pkl',
             '/40.pkl', '/41.pkl','/42.pkl','/43.pkl','/44.pkl','/45.pkl','/46.pkl','/47.pkl', '/48.pkl','/49.pkl',
             '/50.pkl', '/51.pkl','/52.pkl','/53.pkl','/54.pkl','/55.pkl','/56.pkl','/57.pkl', '/58.pkl','/59.pkl',
             '/60.pkl', '/61.pkl','/62.pkl','/63.pkl','/64.pkl','/65.pkl','/66.pkl','/67.pkl', '/68.pkl','/69.pkl',
             '/70.pkl', '/71.pkl','/72.pkl','/73.pkl','/74.pkl','/75.pkl','/76.pkl','/77.pkl', '/78.pkl','/79.pkl']

for annotationFile in annotationList:
    pfile = "/content/drive/MyDrive/Truckin/001/annotations/cuboids" + annotationFile
    with open(pfile, "rb") as fin:
        data = pickle.load(fin)

    #data.info()

    #print(data.label)

    print(f"X: {data['position.x']}, Y: {data['position.y']}, Z: {data['position.z']}, Label: {data.label}") 
    
    """xyz = data[['x', 'y', 'z']].to_numpy()
    print(xyz)
    print(xyz.shape)


    xyz_torch = torch.from_numpy(xyz)
    print(xyz_torch)
    print(type(xyz_torch))
    print(xyz_torch.size())"""
    

PCA to scale dimension from X , 3 to 150,000 , 3 for uniformity among the different times and scenes.

In [ ]:
print(xyz_torch)
print(xyz_torch.size())

def PCA_svd(X, k, center=True):
  n = X.size()[0]
  ones = torch.ones(n).view([n,1])
  h = ((1/n) * torch.mm(ones, ones.t())) if center  else torch.zeros(n*n).view([n,n])
  H = torch.eye(n) - h
  X_center =  torch.mm(H.double(), X.double())
  u, s, v = torch.svd(X_center) 
  components  = v[:k].t()
  explained_variance = torch.mul(s[:k], s[:k])/(n-1)
  return { 'X':X, 'k':k, 'components':components,     
    'explained_variance':explained_variance }

value = PCA_svd(xyz_torch, 3)
print(value)


Camera Data

In [ ]:
from pandaset import DataSet

dataset = DataSet('/content/drive/My Drive/Truckin/')

seq002 = dataset['002']

seq002.load()

print(seq002.__dict__)

cameraObj = seq002.camera

print(cameraObj)

#print(f"Time stamps from lidar object: {cameraObj.timestamps}")

#print(f"Panda set object: {pandaset.sensors.Lidar}")

#print(f"Lidar data obj {cameraObj.data}")

import torch
from torchvision import transforms
import numpy as np
import pickle
from PIL import Image


cameraList = ['/00.jpg', '/01.jpg','/02.jpg','/03.jpg','/04.jpg','/05.jpg','/06.jpg','/07.jpg', 
             '/08.jpg','/09.jpg','/10.jpg','/11.jpg', '/11.jpg','/12.jpg','/13.jpg','/14.jpg',
             '/15.jpg','/16.jpg','/17.jpg','/18.jpg','/19.jpg','/20.jpg','/21.jpg','/22.jpg',
             '/23.jpg','/24.jpg', '/25.jpg','/26.jpg','/27.jpg','/28.jpg','/29.jpg', 
             '/30.jpg', '/31.jpg','/32.jpg','/33.jpg','/34.jpg','/35.jpg','/36.jpg','/37.jpg', '/38.jpg','/39.jpg',
             '/40.jpg', '/41.jpg','/42.jpg','/43.jpg','/44.jpg','/45.jpg','/46.jpg','/47.jpg', '/48.jpg','/49.jpg',
             '/50.jpg', '/51.jpg','/52.jpg','/53.jpg','/54.jpg','/55.jpg','/56.jpg','/57.jpg', '/58.jpg','/59.jpg',
             '/60.jpg', '/61.jpg','/62.jpg','/63.jpg','/64.jpg','/65.jpg','/66.jpg','/67.jpg', '/68.jpg','/69.jpg',
             '/70.jpg', '/71.jpg','/72.jpg','/73.jpg','/74.jpg','/75.jpg','/76.jpg','/77.jpg', '/78.jpg','/79.jpg']

for cameraFile in cameraList:
    cfile = "/content/drive/MyDrive/Truckin/001/camera/front_camera" + cameraFile
    #with open(pfile, "rb") as fin:
    img = Image.open(cfile)

    #data.info()

    convert_tensor = transforms.ToTensor()

    img_tensor = convert_tensor(img)

    print(img_tensor)
    #xyz = data[['x', 'y', 'z']].to_numpy()
    #print(xyz)
    print(f"Shape: {img_tensor.shape}")


    #xyz_torch = torch.from_numpy(xyz)
    #print(xyz_torch)
    #print(type(xyz_torch))
    #print(xyz_torch.size())
    

Lidar Only CNN


In [ ]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_, zeros_, kaiming_normal_


class PointNetfeat(nn.Module):
    def __init__(self, pts_dim, x=1):
        super(PointNetfeat, self).__init__()
        self.output_channel = 512 * x
        self.conv1 = torch.nn.Conv1d(pts_dim, 64 * x, 1)
        self.conv2 = torch.nn.Conv1d(64 * x, 128 * x, 1)
        self.conv3 = torch.nn.Conv1d(128 * x, self.output_channel, 1)
        self.bn1 = nn.BatchNorm1d(64 * x)
        self.bn2 = nn.BatchNorm1d(128 * x)
        self.bn3 = nn.BatchNorm1d(self.output_channel)


    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x)) # NOTE: should not put a relu
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, self.output_channel)
        return x


class PointNet(nn.Module):
    def __init__(self, pts_dim, x, CLS_NUM):
        super(PointNet, self).__init__()
        self.feat = PointNetfeat(pts_dim, x)
        self.fc1 = nn.Linear(512 * x, 256 * x)
        self.fc2 = nn.Linear(256 * x, 256)

        self.pre_bn = nn.BatchNorm1d(pts_dim)
        self.bn1 = nn.BatchNorm1d(256 * x)
        self.bn2 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        # NOTE: should there put a BN?
        self.fc_s1 = nn.Linear(256, 256)
        self.fc_s2 = nn.Linear(256, 3, bias=False)
        self.fc_c1 = nn.Linear(256, 256)
        self.fc_c2 = nn.Linear(256, CLS_NUM, bias=False)
        self.fc_ce1 = nn.Linear(256, 256)
        self.fc_ce2 = nn.Linear(256, 3, bias=False)
        self.fc_hr1 = nn.Linear(256, 256)
        self.fc_hr2 = nn.Linear(256, 1, bias=False)

    def forward(self, x, pred_bbox):
        x = self.feat(self.pre_bn(x))
        x = F.relu(self.bn1(self.fc1(x)))
        feat = F.relu(self.bn2(self.fc2(x)))

        x = F.relu(self.fc_c1(feat))
        logits = self.fc_c2(x)

        x = F.relu(self.fc_ce1(feat))
        centers = self.fc_ce2(x)

        x = F.relu(self.fc_s1(feat))
        sizes = self.fc_s2(x)

        x = F.relu(self.fc_hr1(feat))
        headings = self.fc_hr2(x)

        return logits, centers, sizes, headings

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
                kaiming_normal_(m.weight.data)
                if m.bias is not None:
                    zeros_(m.bias)



Image CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

#Loss and Optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')



Both CNN